# DB Project, data prep #1

## Feature Generation, Data Merging

In [1]:
# Importing relevant packages, loading relevant data.

import numpy as np
import pandas as pd
import geopandas as gpd
from geopandas import GeoDataFrame
from shapely.geometry import Point
import warnings
warnings.filterwarnings('ignore')

crime_df = pd.read_csv('../raw_data/Crimes_-_2015_to_Present.csv')
crime_class_df = pd.read_csv('../raw_data/Crimes_classes.csv')
beats_df = pd.read_csv('../intermediate_data/beats.csv')
weather_df = pd.read_csv('../raw_data/weather_2015-2020.csv')

In [2]:
# Transforming crime data to datetime, indexing by such, limiting data to 2015-2020.

crime_df['Date'] = pd.to_datetime(crime_df['Date'])
crime_df = crime_df.set_index('Date')
crime_df = crime_df['20150101' : '20201231']

In [3]:
# Creating features for month, week of the current year, day of the week, and hour of day .

crime_df['Month'] = crime_df.index.month
crime_df['Week'] = crime_df.index.week
crime_df['Day'] = crime_df.index.day
crime_df['Hour'] = crime_df.index.hour

def find_watch(n):
    if 0 <= n < 8:
        return 'First'
    elif 8 <= n < 16:
        return 'Second'
    elif 16 <= n:
        return 'Third'
        

crime_df['Watch'] = crime_df['Hour'].apply(find_watch)

crime_df.index = crime_df.index.date

In [4]:
# Dropping non-criminal reports

crime_df = crime_df[~crime_df['IUCR'].isin(['5073', '5093', '5094', '5113', '5114'])]

In [5]:
# Transforming weather data to datetime, indexing by such.

weather_df['Date'] = pd.to_datetime(weather_df['DATE'])
weather_df = weather_df.set_index('Date')

In [6]:
# Merging weather data to crime data.

crime_df = pd.merge(crime_df, weather_df, how = 'left', left_index = True, right_index = True)

In [7]:
# Saving total tempral data for analysis

crime_df.to_csv("../intermediate_data/df_temporal_to_graph.csv")

In [8]:
# Preparing crime classes to merge into crime data, and merging.

for index, row in crime_class_df.iterrows():
    if len(row['crime_id']) == 3:
        row['crime_id'] = '0' + row['crime_id']

crime_df = pd.merge(crime_df, crime_class_df, how = 'left', left_on = 'IUCR', right_on = 'crime_id')        

In [9]:
# Merging beat data to crime data.  

crime_df = pd.merge(crime_df, beats_df, how = 'left', left_on = 'Beat', right_on = 'beat_num')

In [10]:
crime_df.loc[crime_df.category_1 == 'serious', 'serious'] = 1
crime_df.serious.fillna(0, inplace=True)

In [11]:
# Dropping undesired features.

col_list = ['ID','Arrest', 'Domestic', 'Beat', 'Year', 'Month', 'Watch', 
            'PRCP', 'SNOW','TMAX', 'TMIN', 'serious']

df_filtered = crime_df.filter(col_list)

In [12]:
df_filtered

,ID,Arrest,Domestic,Beat,Year,Month,Watch,PRCP,SNOW,TMAX,TMIN,serious
0,10225520,False,False,411,2015,1,First,0.0,0.0,32,17,1.0
1,11028448,True,True,1532,2015,1,First,0.0,0.0,32,17,1.0
2,10225760,False,False,2024,2015,1,First,0.0,0.0,32,17,1.0
3,11242929,False,False,223,2015,1,First,0.0,0.0,32,17,1.0
4,10229179,False,False,214,2015,1,First,0.0,0.0,32,17,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1542399,12259610,False,False,322,2020,12,Third,0.0,0.0,30,23,0.0
1542400,12263896,False,False,223,2020,12,Third,0.0,0.0,30,23,1.0
1542401,12274737,False,False,114,2020,12,Third,0.0,0.0,30,23,1.0
1542402,12258658,False,False,1021,2020,12,Third,0.0,0.0,30,23,1.0


In [13]:
groups = df_filtered[['Year','Month','Watch','Beat']]

In [14]:
groups = groups.merge(df_filtered.groupby(['Year','Month', 'Watch','Beat'])['ID'].count().reset_index()).drop_duplicates()

groups = groups.merge(df_filtered.groupby(['Year','Month', 'Watch','Beat'])['serious'].sum().reset_index())
groups = groups.merge(df_filtered.groupby(['Year','Month', 'Watch','Beat'])['Arrest'].sum().reset_index())
groups = groups.merge(df_filtered.groupby(['Year','Month', 'Watch','Beat'])['Domestic'].sum().reset_index())
groups = groups.merge(df_filtered.groupby(['Year','Month', 'Watch','Beat'])['TMAX'].mean().reset_index())
groups = groups.merge(df_filtered.groupby(['Year','Month', 'Watch','Beat'])['TMIN'].mean().reset_index())
groups = groups.merge(df_filtered.groupby(['Year','Month', 'Watch','Beat'])['PRCP'].mean().reset_index())
groups = groups.merge(df_filtered.groupby(['Year','Month', 'Watch','Beat'])['SNOW'].mean().reset_index())

groups.rename({'ID':'Total', 'serious':'Serious'}, axis=1, inplace=True)

In [15]:
groups['rank'] = groups.groupby(['Year','Month'])['Serious'].rank(pct=True)
groups.loc[groups['rank'] > .75, "high_crime"] = 1
groups['high_crime'].fillna(0, inplace=True)
groups.drop('rank', axis = 1, inplace=True)

In [16]:
groups.iloc[1]

Year               2015
Month                 1
Watch             First
Beat               1532
Total                24
Serious            13.0
Arrest                9
Domestic              7
TMAX             31.125
TMIN          16.583333
PRCP           0.045833
SNOW           0.233333
high_crime          0.0
Name: 1, dtype: object

In [17]:
# Merging beat data to crime data.  

crime_df = pd.merge(groups, beats_df, how = 'left', left_on = 'Beat', right_on = 'beat_num')

In [18]:
crime_df.iloc[1]

Year                            2015
Month                              1
Watch                          First
Beat                            1532
Total                             24
Serious                         13.0
Arrest                             9
Domestic                           7
TMAX                          31.125
TMIN                       16.583333
PRCP                        0.045833
SNOW                        0.233333
high_crime                       0.0
Unnamed: 0                        80
beat                               3
beat_num                        1532
district                          15
sector                             3
count_l_stops                    1.0
count_bus_stops                 11.0
count_metra_stops                0.0
count_restaurants               14.0
count_bars                       2.0
count_daycares                   1.0
count_entertainment              0.0
count_businesses                24.0
road_distance_ft       162950.259395
T

In [19]:
crime_df.drop(["Unnamed: 0", 'beat'], axis=1, inplace=True)

In [20]:
crime_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59093 entries, 0 to 59092
Data columns (total 28 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Year                 59093 non-null  int64  
 1   Month                59093 non-null  int64  
 2   Watch                59093 non-null  object 
 3   Beat                 59093 non-null  int64  
 4   Total                59093 non-null  int64  
 5   Serious              59093 non-null  float64
 6   Arrest               59093 non-null  int64  
 7   Domestic             59093 non-null  int64  
 8   TMAX                 59093 non-null  float64
 9   TMIN                 59093 non-null  float64
 10  PRCP                 59093 non-null  float64
 11  SNOW                 59093 non-null  float64
 12  high_crime           59093 non-null  float64
 13  beat_num             59093 non-null  int64  
 14  district             59093 non-null  int64  
 15  sector               59093 non-null 

In [21]:
%run geoprocessing_helpers.ipynb

../raw_data/Boundaries - Police Beats (current).geojson EPSG:3435
../raw_data/Boundaries - Census Blocks - 2010.geojson EPSG:3435


In [22]:
beats = gpd.read_file("../raw_data/Boundaries - Police Beats (current).geojson")

In [23]:
beats

,beat,beat_num,district,sector,geometry
0,1,1713,17,1,"MULTIPOLYGON (((-87.70473 41.97577, -87.70472 ..."
1,0,3100,31,0,"MULTIPOLYGON (((-87.83365 41.97535, -87.83366 ..."
2,5,1651,16,5,"MULTIPOLYGON (((-87.90684 41.97656, -87.91070 ..."
3,1,1914,19,1,"MULTIPOLYGON (((-87.64492 41.96973, -87.64431 ..."
4,1,1915,19,1,"MULTIPOLYGON (((-87.63724 41.96599, -87.63644 ..."
...,...,...,...,...,...
272,1,0314,03,1,"MULTIPOLYGON (((-87.58879 41.78612, -87.58844 ..."
273,2,0825,08,2,"MULTIPOLYGON (((-87.67917 41.78656, -87.67912 ..."
274,1,0313,03,1,"MULTIPOLYGON (((-87.60854 41.78583, -87.60808 ..."
275,2,0823,08,2,"MULTIPOLYGON (((-87.69354 41.78458, -87.69354 ..."


In [24]:
crime_df["Beat"]=crime_df["Beat"].astype("string")
crime_df["Beat"] =crime_df["Beat"].str.zfill(4)

In [25]:
crime_df.iloc[0]

Year                            2015
Month                              1
Watch                          First
Beat                            0411
Total                             16
Serious                          9.0
Arrest                             3
Domestic                           5
TMAX                         32.5625
TMIN                            21.5
PRCP                        0.055625
SNOW                        0.326667
high_crime                       0.0
beat_num                         411
district                           4
sector                             1
count_l_stops                    0.0
count_bus_stops                 49.0
count_metra_stops                2.0
count_restaurants               16.0
count_bars                       0.0
count_daycares                   1.0
count_entertainment              0.0
count_businesses                58.0
road_distance_ft       231176.656022
TOTAL POPULATION         5470.665022
dist_to_police           7180.695576
d

In [26]:
beats.iloc[1]

beat                                                        0
beat_num                                                 3100
district                                                   31
sector                                                      0
geometry    (POLYGON ((-87.83365455041093 41.9753548151922...
Name: 1, dtype: object

In [27]:
crime_data = beats.merge(crime_df, left_on="beat_num", right_on="Beat")

In [28]:
crime_data = compute_spatial_lag(crime_data, "high_crime", 
                                 "high_crime_geog_lag", 
                                 "Year")

In [29]:
crime_data.iloc[1]

beat                                                                   1
beat_num_x                                                          1713
district_x                                                            17
sector_x                                                               1
geometry               (POLYGON ((-87.7047252651434 41.97577443021446...
Year                                                                2015
Month                                                                  1
Watch                                                             Second
Beat                                                                1713
Total                                                                 20
Serious                                                             13.0
Arrest                                                                 5
Domestic                                                               4
TMAX                                               

In [30]:
crime_data.drop(['beat_num_y', 'district_y', 'sector_y'], axis=1, inplace = True)
crime_data.rename(columns={'Total': 'Crimes',
                         'beat_num_x': 'beat_num', 
                         'district_x': 'district', 
                         'sector_x': 'sector'}, inplace=True)

In [31]:
crime_data.columns

Index(['beat', 'beat_num', 'district', 'sector', 'geometry', 'Year', 'Month',
       'Watch', 'Beat', 'Crimes', 'Serious', 'Arrest', 'Domestic', 'TMAX',
       'TMIN', 'PRCP', 'SNOW', 'high_crime', 'count_l_stops',
       'count_bus_stops', 'count_metra_stops', 'count_restaurants',
       'count_bars', 'count_daycares', 'count_entertainment',
       'count_businesses', 'road_distance_ft', 'TOTAL POPULATION',
       'dist_to_police', 'dist_to_hospital', 'high_crime_geog_lag'],
      dtype='object')

In [32]:
# Saving aggregate geospatial data for analysis

crime_data.to_csv("../intermediate_data/df_geospatial_to_graph.csv")

In [33]:
crime_data.drop(columns = "geometry", axis=0, inplace=True)

In [34]:
crime_data.to_csv("../intermediate_data/high_crime_labeled.csv")